In [0]:
# =====================================
# WIDGETS
# =====================================
dbutils.widgets.text(
    name="config_path",
    defaultValue="/Workspace/Users/ud3041@gmail.com/end-to-end-ETL-pipeline/medallion/gold/config.json",
    label="Config File Path"
)

# =====================================
# IMPORTS
# =====================================
import json
from utils.logger import get_logger
from utils.sparksession import create_spark_session

# =====================================
# INITIALISE LOGGER & SPARK
# =====================================
logger = get_logger("ds2g_silver_to_gold")
spark = create_spark_session("DS2G | Silver to Gold Promotion")

logger.info("Spark session initialised")

# =====================================
# LOAD CONFIG
# =====================================
config_path = dbutils.widgets.get("config_path")
logger.info(f"Loading config from: {config_path}")

with open(config_path, "r") as f:
    config = json.load(f)

CATALOG = config["catalog"]
SILVER_SCHEMA = config["silver_schema"]
GOLD_SCHEMA = config["gold_schema"]
TABLES = config["tables"]

logger.info(
    f"Config loaded | Catalog={CATALOG}, Silver={SILVER_SCHEMA}, Gold={GOLD_SCHEMA}"
)
logger.info(f"Tables to promote: {TABLES}")

# =====================================
# SILVER → GOLD PROMOTION
# =====================================
for table in TABLES:
    logger.info(f"Promoting table: {table}")

    silver_df = spark.table(f"`{CATALOG}`.{SILVER_SCHEMA}.{table}")

    # Drop technical columns not required in Gold
    silver_df = silver_df.drop("file_path", "last_updated_ts")

    (
        silver_df.write
        .format("delta")
        .mode("overwrite")
        .option("overwriteSchema", "true")
        .saveAsTable(f"`{CATALOG}`.{GOLD_SCHEMA}.{table}")
    )

    logger.info(f"Table promoted successfully: {table}")

logger.info("Silver → Gold promotion completed successfully")